<a href="https://colab.research.google.com/github/MihaelOne/webscraper-json-tester/blob/main/webscraper_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 lxml pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from google.colab import files
import re

print("⬆️ Upload your Web Scraper .json file for testing")
uploaded = files.upload()
json_path = next(iter(uploaded))
print(f"✔️ JSON detected: {json_path}")

with open(json_path, encoding='utf-8') as f:
    sitemap = json.load(f)

print(f"\n=== TESTING SITEMAP: {sitemap.get('_id', '?')} ===\n")

urls = sitemap['startUrl']
selectors = sitemap['selectors']

results = []

def bs_contains(soup, selector):
    """
    Support for WebScraper :contains("text") pseudo-class
    """
    match = re.match(r'(.*?)\:contains\([\'"]?(.*?)[\'"]?\)$', selector)
    if not match:
        return None

    tag = match.group(1).strip() or True
    text = match.group(2).strip()

    elems = soup.find_all(tag if tag != '' else True)
    for el in elems:
        if text.lower() in el.get_text(strip=True).lower():
            return el.get_text(strip=True)
    return None


for url in urls:
    print(f"\n🔗 Loading: {url}")
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, headers=headers, timeout=15)
        soup = BeautifulSoup(r.content, 'lxml')
        row = {'url': url}

        for sel in selectors:
            name = sel['id']
            selector = sel['selector']

            found = None

            # 1. check :contains() first
            if ":contains" in selector:
                found = bs_contains(soup, selector)

            # 2. if not :contains, use normal CSS selector
            if not found:
                try:
                    el = soup.select_one(selector)
                    found = el.get_text(strip=True) if el else None
                except:
                    found = None

            row[name] = found if found else "(not found)"
            print(f"  {name}: {row[name]}")

        results.append(row)

    except Exception as e:
        print("  Error:", e)
        results.append({'url': url, 'error': str(e)})

df = pd.DataFrame(results)
df.to_csv("json_test_results.csv", index=False)
print("\n✅ Test completed! Results saved as 'json_test_results.csv'")
df


⬆️ Upload your Web Scraper .json file for testing
